In [1]:
# rtmidi on Windows — loopMIDI detection test
# Run with loopMIDI open and at least one port created (e.g. "loopMIDI 1")

import sys
print(f"Python: {sys.version}")
print(f"Platform: {sys.platform}")

Python: 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]
Platform: win32


In [2]:
# 1) Check we have python-rtmidi (not the wrong "rtmidi" package)
try:
    import rtmidi
    print("rtmidi imported OK")
    print(f"  rtmidi.__file__ = {getattr(rtmidi, '__file__', 'N/A')}")
    has_midiin = hasattr(rtmidi, "MidiIn")
    print(f"  has MidiIn: {has_midiin}")
    if not has_midiin:
        print("  -> Wrong package? Uninstall 'rtmidi' and install 'python-rtmidi'")
except Exception as e:
    print(f"Import failed: {e}")
    print("Install: pip install python-rtmidi")

rtmidi imported OK
  rtmidi.__file__ = c:\Users\john\OneDrive\Documents\winprojects\psiwave-matrix\.venv\Lib\site-packages\rtmidi\__init__.py
  has MidiIn: True


In [3]:
# 2) Which APIs is rtmidi compiled with? (Windows needs API_WINDOWS_MM for loopMIDI)
import rtmidi

get_apis = getattr(rtmidi, "get_compiled_api", None)
if get_apis and callable(get_apis):
    apis = get_apis()
    print("Compiled APIs:", apis)
    # API_WINDOWS_MM = 0 on Windows, needed for virtual ports like loopMIDI
    WMM = getattr(rtmidi, "API_WINDOWS_MM", None)
    if WMM is not None:
        print(f"  API_WINDOWS_MM = {WMM}, available: {WMM in apis if apis else 'N/A'}")
else:
    print("get_compiled_api not found")

Compiled APIs: [4]
  API_WINDOWS_MM = 4, available: True


In [4]:
# 3) List ports WITH default API (often misses loopMIDI on Windows)
import rtmidi

midi_in_default = rtmidi.MidiIn()
ports_default = midi_in_default.get_ports()
print("MidiIn (default API) — get_ports():")
if not ports_default:
    print("  (none)")
else:
    for i, name in enumerate(ports_default):
        print(f"  {i}: {name}")
del midi_in_default

MidiIn (default API) — get_ports():
  0: gandalf 0
  1: test123 1


In [5]:
# 4) List ports WITH Windows MM API (required for loopMIDI on Windows)
import rtmidi

api = getattr(rtmidi, "API_WINDOWS_MM", None)
get_apis = getattr(rtmidi, "get_compiled_api", None)
if api is not None and get_apis and api in get_apis():
    midi_in_mm = rtmidi.MidiIn(rtapi=api)
    ports_mm = midi_in_mm.get_ports()
    print("MidiIn (Windows MM API) — get_ports():")
    if not ports_mm:
        print("  (none) — create a port in loopMIDI and try again")
    else:
        for i, name in enumerate(ports_mm):
            print(f"  {i}: {name}")
    del midi_in_mm
else:
    print("Windows MM API not available in this build")

MidiIn (Windows MM API) — get_ports():
  0: gandalf 0
  1: test123 1


In [6]:
# 5) Open first input port (Windows MM) and read messages for a few seconds
import rtmidi
import time

api = getattr(rtmidi, "API_WINDOWS_MM", None)
get_apis = getattr(rtmidi, "get_compiled_api", None)
if api is None or not get_apis or api not in get_apis():
    print("Skipping: Windows MM API not available")
else:
    midi_in = rtmidi.MidiIn(rtapi=api)
    ports = midi_in.get_ports()
    if not ports:
        print("No ports — create a loopMIDI port and re-run the previous cell")
    else:
        midi_in.open_port(0)
        print(f"Opened: {ports[0]}")
        print("Listening for 5 seconds... (send MIDI to this port from another app)")
        t0 = time.perf_counter()
        while time.perf_counter() - t0 < 5:
            msg = midi_in.get_message()
            if msg:
                print(f"  {msg}")
            time.sleep(0.01)
        midi_in.close_port()
        del midi_in
        print("Done.")

Opened: gandalf 0
Listening for 5 seconds... (send MIDI to this port from another app)
Done.


In [7]:
# 6) Quick combined diagnosis (re-run this after changing loopMIDI settings)
import rtmidi
import time

api = getattr(rtmidi, "API_WINDOWS_MM", None)
if api is None:
    print("No API_WINDOWS_MM constant in rtmidi build")
else:
    mi = rtmidi.MidiIn(rtapi=api)
    mo = rtmidi.MidiOut(rtapi=api)

    # Give Windows/driver a moment in case a port was just created.
    for t in (0.0, 0.6, 1.2):
        if t:
            time.sleep(t)
        in_ports = mi.get_ports()
        out_ports = mo.get_ports()
        print(f"t+{t:.1f}s  in={in_ports}")
        print(f"t+{t:.1f}s out={out_ports}")

    all_names = [n.lower() for n in (in_ports + out_ports)]
    if any("loop" in n for n in all_names):
        print("loopMIDI-like port detected")
    else:
        print("No loopMIDI-like name detected. In loopMIDI app, click '+' to create a port, then re-run.")

    del mi
    del mo

t+0.0s  in=['gandalf 0', 'test123 1']
t+0.0s out=['Microsoft GS Wavetable Synth 0', 'gandalf 1', 'test123 2']
t+0.6s  in=['gandalf 0', 'test123 1']
t+0.6s out=['Microsoft GS Wavetable Synth 0', 'gandalf 1', 'test123 2']
t+1.2s  in=['gandalf 0', 'test123 1']
t+1.2s out=['Microsoft GS Wavetable Synth 0', 'gandalf 1', 'test123 2']
No loopMIDI-like name detected. In loopMIDI app, click '+' to create a port, then re-run.


In [8]:
# 7) Try opening a specific port name: "psiwave"
import rtmidi
import time

TARGET = "psiwave"
api = getattr(rtmidi, "API_WINDOWS_MM", None)
get_apis = getattr(rtmidi, "get_compiled_api", None)

if api is None or not get_apis or api not in get_apis():
    print("Windows MM API not available in this rtmidi build")
else:
    midi_in = rtmidi.MidiIn(rtapi=api)
    ports = midi_in.get_ports()

    print("Available input ports:")
    if not ports:
        print("  (none)")
    else:
        for i, name in enumerate(ports):
            print(f"  {i}: {name}")

    # Case-insensitive contains match on "psiwave".
    matches = [(i, name) for i, name in enumerate(ports) if TARGET in name.lower()]

    if not matches:
        print(f"No input port containing '{TARGET}' found.")
        print("Create/rename a loopMIDI port to include 'psiwave', then re-run.")
    else:
        idx, name = matches[0]
        print(f"Opening input port {idx}: {name}")
        midi_in.open_port(idx)

        print("Listening for 5 seconds... send MIDI to this port now.")
        t0 = time.perf_counter()
        saw_any = False
        while time.perf_counter() - t0 < 5.0:
            msg = midi_in.get_message()
            if msg:
                saw_any = True
                print(" ", msg)
            time.sleep(0.01)

        midi_in.close_port()
        del midi_in

        if not saw_any:
            print("No MIDI messages received during test window.")
            print("Verify another app/device is routing MIDI into this input port.")
        else:
            print("Received MIDI data from psiwave-matched port.")

Available input ports:
  0: gandalf 0
  1: test123 1
No input port containing 'psiwave' found.
Create/rename a loopMIDI port to include 'psiwave', then re-run.
